In [1]:
import wandb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling

sns.set()

In [2]:
run = wandb.init(project="nyc_airbnb", save_code=True)

wandb: Currently logged in as: ibrahimsherif (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
artifact = run.use_artifact("nyc_airbnb/raw_data.csv:latest\")\n",
df = pd.read_csv(artifact.file())

df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [4]:
profile = pandas_profiling.ProfileReport(df, title="Pandas Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,2.000000e+04,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,15877.000000,20000.000000,20000.000000
mean,1.892380e+07,6.746034e+07,40.728455,-73.952125,153.269050,6.992100,23.274100,1.377446,6.955450,112.901200
std,1.101223e+07,7.857936e+07,0.054755,0.046559,243.325609,21.645449,44.927793,1.683006,32.433831,131.762226
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.393540e+06,7.853718e+06,40.689420,-73.983030,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.952117e+07,3.111431e+07,40.722730,-73.955640,105.000000,2.000000,5.000000,0.720000,1.000000,44.000000
75%,2.912936e+07,1.068426e+08,40.762990,-73.936380,175.000000,5.000000,23.000000,2.010000,2.000000,229.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,10000.000000,1250.000000,607.000000,27.950000,327.000000,365.000000


In [6]:
import pandas as pd
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [7]:
data = {
    "N": [0, 1, 2, 3, 4, 5, 6, 7],
    "Issue_Key": ['JIRA_1', 'JIRA_2', 'JIRA_3', 'JIRA_4', 'JIRA_5', 'JIRA_6', 'JIRA_7', 'JIRA_8'],
    "Request_Type": ['Bug', 'Improvement', 'Improvement', 'Improvement', 'Improvement', 'Improvement', 'Bug', 'Bug'],
    "Assignee_Name": ['User1', 'User2', 'User3', 'User3', 'User3', 'User3', 'User5', 'User5'],
    "Status": ['Awaiting Review', 'In Progress', 'New', 'Closed', 'Awaiting Review', 'New', 'Closed', 'Closed']
}

pd.DataFrame(data)

,N,Issue_Key,Request_Type,Assignee_Name,Status
0,0,JIRA_1,Bug,User1,Awaiting Review
1,1,JIRA_2,Improvement,User2,In Progress
2,2,JIRA_3,Improvement,User3,New
3,3,JIRA_4,Improvement,User3,Closed
4,4,JIRA_5,Improvement,User3,Awaiting Review
5,5,JIRA_6,Improvement,User3,New
6,6,JIRA_7,Bug,User5,Closed
7,7,JIRA_8,Bug,User5,Closed
